# Group Normalization解析

## group normalization

论文地址：https://arxiv.org/abs/1803.08494

在之前的文章中有介绍过[Batch Normalization](https://github.com/MorvanLi/deepLearningTutorial/blob/main/batchNormalization.ipynb)，以及[Layer Normalization](https://github.com/MorvanLi/deepLearningTutorial/blob/main/layerNormalization.ipynb)。今天来简单聊聊GN(Group Normalization)。在视觉领域，其实最常用的还是BN，但BN也有缺点，通常需要比较大的Batch Size。如下图所示，蓝色的线代表BN，当batch size小于16后error明显升高（但大于16后的效果确实要更好）。对于比较大型的网络或者GPU显存不够的情况下，通常无法设置较大的batch size，此时可以使用GN。如下图所示，batch size的大小对GN并没有影响，所以当**batch size设置较小时，可以采用GN**。

![jupyter](./images/groupNormalization_img1.png) 

无论是BN、LN还是GN，公式都是一样的，都是减均值$E(x)$，除以标准差$\sqrt{Var(x)+\epsilon} $，其中$\epsilon$是一个非常小的数字默认$10^{-5}$，是为了防止分母为零。以及两个可训练的参数$\beta$,$\gamma$。不同在于是在哪个/哪些维度上进行操作：
$$y =\frac{x-E(x)}{\sqrt{Var(x)+\epsilon } } $$
对于GN(Group Normalization)的操作如下图所示，假设$num\_groups=2$原论文中默认为32，由于和batch_size无关，我们直接看对于一个样本的情况。假设某层输出得到$x$，根据$num\_groups$沿$channel$方向均分成$num\_groups$份，然后对每一份求均值和方差，接着按照上面的公式进行计算即可，非常简单。

![jupyter](./images/groupNormalization_img2.png) 

# pytorch实验

In [9]:
import torch
import torch.nn as nn


def group_norm(x: torch.Tensor,
               num_groups: int,
               num_channels: int,
               eps: float = 1e-5,
               gamma: float = 1.0,
               beta: float = 0.):
    # divmod(a,b)方法返回的是a/b（取整）以及a对b的余数
    assert divmod(num_channels, num_groups)[1] == 0
    channels_per_group = num_channels // num_groups

    new_tensor = []
    for t in x.split(channels_per_group, dim=1):
        var_mean = torch.var_mean(t, dim=[1, 2, 3], unbiased=False)
        var = var_mean[0]
        mean = var_mean[1]
        t = (t - mean[:, None, None, None]) / torch.sqrt(var[:, None, None, None] + eps)
        t = t * gamma + beta
        new_tensor.append(t)

    new_tensor = torch.cat(new_tensor, dim=1)
    return new_tensor


torch.manual_seed(0)
num_groups = 2
num_channels = 4
eps = 1e-5

img = torch.rand(2, num_channels, 2, 2)
print(f"原始数据: \n {img}")

gn = nn.GroupNorm(num_groups=num_groups, num_channels=num_channels, eps=eps)
r1 = gn(img)
print(f"官方结果: \n {r1}")

r2 = group_norm(img, num_groups, num_channels, eps)
print(f"实验结果: \n {r2}")


原始数据: 
 tensor([[[[0.4963, 0.7682],
          [0.0885, 0.1320]],

         [[0.3074, 0.6341],
          [0.4901, 0.8964]],

         [[0.4556, 0.6323],
          [0.3489, 0.4017]],

         [[0.0223, 0.1689],
          [0.2939, 0.5185]]],


        [[[0.6977, 0.8000],
          [0.1610, 0.2823]],

         [[0.6816, 0.9152],
          [0.3971, 0.8742]],

         [[0.4194, 0.5529],
          [0.9527, 0.0362]],

         [[0.1852, 0.3734],
          [0.3051, 0.9320]]]])
官方结果: 
 tensor([[[[ 0.0726,  1.0785],
          [-1.4357, -1.2746]],

         [[-0.6259,  0.5824],
          [ 0.0498,  1.5528]],

         [[ 0.5513,  1.5218],
          [-0.0350,  0.2552]],

         [[-1.8289, -1.0240],
          [-0.3372,  0.8968]]],


        [[[ 0.3631,  0.7480],
          [-1.6552, -1.1992]],

         [[ 0.3027,  1.1812],
          [-0.7673,  1.0268]],

         [[-0.1627,  0.2699],
          [ 1.5656, -1.4046]],

         [[-0.9216, -0.3118],
          [-0.5331,  1.4984]]]], grad_fn=<NativeGro